In [3]:
import numpy as np
import torch
from torch.autograd import grad
import torch.nn as nn
from numpy import genfromtxt
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

tSI_data = genfromtxt('tSI_data.csv', delimiter=',') #in the form of [t, S, I]
tao_data = genfromtxt('tao_data.csv', delimiter=',') #in the form of [t, tao_star, T_star, u]

In [11]:
class DINN:
    def __init__(self, t, S_data, I_data, tao_data, T_star_data, u_data): #t, S_data, I_data, tao_data, T_star_data, u_data
        
        #data
        self.t = torch.tensor(t, requires_grad = True).float()
        self.S = torch.tensor(S_data, requires_grad = True)
        self.I = torch.tensor(I_data, requires_grad = True)
        self.taos = torch.tensor(tao_data, requires_grad = True)
        self.T_stars = torch.tensor(T_star_data, requires_grad = True)
        self.u = torch.tensor(u_data, requires_grad = True).float() 
        self.t_u = torch.stack((self.t, self.u), dim=1) #stack for the NN input 

        #NN init
        self.neural_network = self.neural_network()
        
        #learnable parameters
        self.alpha1 = torch.nn.Parameter(torch.rand(len(t), requires_grad=True))
        self.alpha2 = torch.nn.Parameter(torch.rand(len(t), requires_grad=True))
        self.mu = torch.nn.Parameter(torch.rand(len(t), requires_grad=True))
        self.beta = torch.nn.Parameter(torch.rand(len(t), requires_grad=True))

        #combine parameters for optimizer
        self.params = list(self.neural_network.parameters())
        self.params.extend(list([self.alpha1, self.alpha2, self.mu, self.beta]))


        #predictions
        #self.S_pred, self.I_pred, self.tao_pred, self.T_star_pred = self.net_x(self.t_u) 
        #self.f1, self.f2 = self.net_f(self.t, self.u)
        
        #loss
        #self.loss = (torch.mean(torch.square(self.S - self.S_pred))+torch.mean(torch.square(self.I - self.I_pred)) +
        #        torch.mean(torch.square(self.f1)) + torch.mean(torch.square(self.f2)) \
        #        + torch.mean(torch.square(self.taos - self.tao_pred)) + torch.mean(torch.square(self.T_stars - self.T_star_pred)))

        #optimizer
        self.learning_rate = 0.01
        self.optimizer = optim.Adam(self.params, lr = self.learning_rate)


    # NN
    class neural_network(nn.Module):
        def __init__(self):
            super(DINN.neural_network, self).__init__()
            self.fc1=nn.Linear(2, 20) #takes [t, u]
            self.fc2=nn.Linear(20, 20)
            self.out=nn.Linear(20, 4) #outputs S, I, tao, T*

        def forward(self, x):
            x=F.relu(self.fc1(x))
            x=F.relu(self.fc2(x))
            x=self.out(x)
            return x    
    # Net x
    def net_x(self, t_u):
        net_output = self.neural_network(t_u)
        S_pred = net_output[:,0]
        I_pred = net_output[:,1]
        tao_pred = net_output[:,2]
        T_star_pred = net_output[:,3]
        return S_pred, I_pred, tao_pred, T_star_pred

    # Net f
    def net_f(self, t, u):
        alpha1 = self.alpha1
        alpha2 = self.alpha2
        mu = self.mu
        beta = self.beta

        S_pred, I_pred, tao_pred, T_star_pred = self.net_x(torch.stack((t, u), dim=1))

        #need to fix this loop to be more efficient (this just calculates S_t, I_t)
        S_t_list = []
        I_t_list = [] 
        for input_tensor in self.t_u:
            output_tensor = self.neural_network(input_tensor) #size 4 (S_pred, I_pred, tao_pred, T_star_pred)
            S_t_value = grad(output_tensor[0], input_tensor, retain_graph=True)[0][0] #derivative of S wrt t            
            I_t_value = grad(output_tensor[1], input_tensor, retain_graph=True)[0][0] #derivative of I wrt t
            S_t_list.append(S_t_value)
            I_t_list.append(I_t_value)
        #convert to tensors
        S_t = torch.tensor(S_t_list, requires_grad = True)
        I_t = torch.tensor(I_t_list, requires_grad = True)        

        f1 = S_t + self.beta * S_pred * I_pred + u * (t > tao_pred) * alpha1
        f2 = I_t - self.beta * S_pred * I_pred + mu * I_pred + u * (t > tao_pred) * alpha2
        return f1, f2
    
    #train    
    def train(self, n_epochs):
        print('\nstarting training...\n')
        print('self.alpha1[0] before training', self.alpha1[0])

        losses = []

        for epoch in range(n_epochs):
            print('epoch: ', epoch)
            self.optimizer.zero_grad()
            
            f1, f2 = self.net_f(self.t, self.u)
            S_pred, I_pred, tao_pred, T_star_pred = self.net_x(self.t_u)

            loss = (torch.mean(torch.square(self.S-S_pred))+torch.mean(torch.square(self.I-I_pred)) \
                    +torch.mean(torch.square(f1)) + torch.mean(torch.square(f2)) \
                    +torch.mean(torch.square(self.taos-tao_pred)) + torch.mean(torch.square(self.T_stars-T_star_pred))) #tao, T_star

            losses.append(loss)
            print('before backward')
            loss.backward()
            print('after backward')
            torch.nn.utils.clip_grad_norm_(self.neural_network.parameters(), 100) #gradient clipping
            self.optimizer.step()

            #print
            if epoch % 1 == 0:
                #print('\nepoch: ', epoch)
                print('loss: ', losses[-1])
                print('\nself.alpha1 after the training: ', self.alpha1[0])

        plt.plot(losses, color = 'teal')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')

dinn = DINN(tSI_data[0], tSI_data[1], tSI_data[2], tao_data[1], tao_data[2], tao_data[3]) #t, S_data, I_data, tao_data, T_star_data, u
dinn.train(10)


starting training...

self.alpha1[0] before training tensor(0.9947, grad_fn=<SelectBackward>)
epoch:  0
before backward
after backward
loss:  tensor(926070.9222, dtype=torch.float64, grad_fn=<AddBackward0>)

self.alpha1 after the training:  tensor(0.9947, grad_fn=<SelectBackward>)
epoch:  1
before backward


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.